# Basic setup

In [3]:
import settings as sets
import gspread
import pandas as pd
import os
import tkinter as tk


# Write there your directory,
# for credentials and CSV export.
# It's /sensitive/ by default.

last_directory = sets.LAST_DIRECTORY

# WARNING ACHTUNG ВНИМАНИЕ
# Don't let this folder get synced
# else it would leak your credentials
# and your users' data.

# Put there your Google Sheets table's name
google_sheet_name = ''

# Import CSV export and credentials

In [4]:
# It scans all files in that folder
# and pick the first match by extension
def import_user_files():
    pass
csv_name = ''
files = os.listdir(directory)

for i in files:
    if i[-3:] == str('csv'):
        csv_name = directory+i
        break
print('loaded:', csv_name)

json_name = ''
for i in files:
    if i[-4:] == str('json'):
        json_name = directory+i
        break
print('loaded:', json_name)

NameError: name 'directory' is not defined

# Getting tables together

In [ ]:
# Pull Google Sheet (should be allowed)
# to compare their lenght
gc = gspread.service_account(filename=json_name)
google_sheet = gc.open(google_sheet_name)
# Notice that it points to sheet1. You can change it.
google_df = pd.DataFrame(google_sheet.sheet1.get_all_values())
google_len = len(google_sheet.sheet1.col_values(1))-1

# Set the first row from GS as column names
google_df.columns = google_df.iloc[0]
google_df = google_df[1:]

# Open CSV to compare their lenght
csv_df = pd.read_csv(csv_name)
csv_len = len(csv_df['Entry ID'])

# Fill NaN and sort values to match
# you can customize this value
fill_with = '-'
google_df, csv_df = google_df.fillna(fill_with), csv_df.fillna(fill_with)
csv_df = csv_df.sort_values(by=['Entry ID'], ascending=True)

# Calculate difference between their lenghtes
print('# of strings in GSS: ', google_len)
print('# of strings in CSV: ', csv_len)
difference = csv_len - google_len

# of strings in GSS:  3
# of strings in CSV:  3


# Select rows

In [ ]:
from xlsxwriter.utility import xl_col_to_name as xlsxed

columns_choice = pd.DataFrame(csv_df.columns, columns=['name'])
display(columns_choice)

,name
0,Entry ID
1,Что для Вас есть авторская песня
2,"Страна, город"
3,Возраст
4,Место учёбы или работы
5,Адрес электронной почты
6,Согласие на обработку персональных данных
7,Custom HTML
8,Custom HTML.1
9,Название номера


In [ ]:
column_1 = int(input('Enter first column:') or 0)
column_2 = int(input('Enter last column:') or 1)

x_col_1 = xlsxed(column_1)
x_col_2 = xlsxed(column_2)

print('Columns', column_1, 'to', column_2, 'are a ', x_col_1+':'+x_col_2, 'range.')

Columns 0 to 1 are a  A:B range.


# Pushing new rows

In [ ]:
# Check if columns' list and labels matches -
# this script would stop otherwise
if csv_df.columns.all() == google_df.columns.all():
    columns_len = len(csv_df.columns.all())
    rows_len = google_len
    # Check if our CSV has new rows over GSheets
    if difference > 0:
        print("They differ by ", difference, ' strings!')

    # If yes, we push new lines to GSheets in a cycle
        current_row = google_len + 1

        for i in range(difference+1):
            taken_row = csv_df.iloc[current_row-2].astype('str')
            
            # constructing an Excel-style pointer
            diff_range = str(x_col_1 + str(current_row) + ':' + x_col_2 + str(current_row))
            
            # it double checks if'd write into an empty row
            if google_sheet.sheet1.cell(current_row,1).value == None:
                google_sheet.sheet1.update(values =
                                        [taken_row.values.tolist()],
                                        range_name = diff_range)
            current_row+=1
        print('Tables are in sync.')
    
    # If it's the oposite or they are equal,
    # nothing would happen   
    elif difference < 0:
        print('Your CSV has less entries than Google Sheets. Wrong CSV?')
    else: print('Tables are in sync. No actions needed.')

else: print ('Column labels don\'t match. Wrong tables?')

Tables are in sync. No actions needed.


## END

In [7]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import settings as sets

window = tk.Tk()

csv_file = tk.StringVar()
csv_file = sets.csv_file

json_file = tk.StringVar()
json_file = sets.json_file

window.resizable(0, 0)
window.title('CSV2GSheets')
window.geometry('800x600')

window.columnconfigure(0, weight=1)
window.columnconfigure(1, weight=3)
window.columnconfigure(2, weight=1)

window.rowconfigure(0, weight=1)
window.rowconfigure(1, weight=1)
window.rowconfigure(2, weight=5)
window.rowconfigure(3, weight=1)

def uploadCSV(event=None):
    filename = filedialog.askopenfilename(
        initialdir=last_directory or 'sensitive/',
        title="Open your CSV file",
        filetypes=(("CSV files", "*.csv"),))
    # csv_file.set(filename)
    window.update()

def uploadJSON(event=None):
    filename = filedialog.askopenfilename(
        initialdir=last_directory or 'sensitive/',
        title="Open your CSV file",
        filetypes=(("CSV files", "*.csv"),))
    # json_file.set(filename)
    window.update()

def CloseWindow():
    window.quit()
    window.destroy()
    
def updateWindow():
    window.update()

upload_csv_lable = ttk.Label(window, text="Your CSV file:")
upload_csv_lable.grid(column=0, row=0, sticky=tk.W, padx=5, pady=5)
upload_csv_line = ttk.Label(window, text=csv_file)
upload_csv_line.grid(column=1, row=0, sticky=tk.W, padx=5, pady=5)
upload_csv_button = tk.Button(window, text='Browse', command=uploadCSV+updateWindow)
upload_csv_button.grid(column=2, row=0, sticky=tk.E, padx=5, pady=5)


upload_google_lable = ttk.Label(window, text="Your JSON file:")
upload_google_lable.grid(column=0, row=1, sticky=tk.W, padx=5, pady=5)
upload_google_line = ttk.Label(window, text=csv_file)
upload_google_line.grid(column=1, row=1, sticky=tk.W, padx=5, pady=5)
upload_google_button = tk.Button(window, text='Browse', command=uploadJSON+updateWindow)
upload_google_button.grid(column=2, row=1, sticky=tk.E, padx=5, pady=5)

close_button = tk.Button(window, text='Close', command=CloseWindow)
close_button.grid(column=3, row=3, sticky=tk.SE, padx=5, pady=5)




window.mainloop()

TypeError: unsupported operand type(s) for +: 'function' and 'function'

In [ ]:
tf = filedialog.askopenfilename(
        initialdir="C:/Users/MainFrame/Desktop/", 
        title="Open Text file", 
        filetypes=(("Text Files", "*.txt"),)
        )

In [ ]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
quit()

NameError: name 'destroy' is not defined

In [ ]:
import tkinter as tk
from tkinter import filedialog as fd

def get_file():
    file = fd.askopenfile()
    if file:
        print(file.name)
        root.destroy(root)

root = tk.Tk

root.button_1 = tk.Button(text="Click me!",
    width=25,
    height=50,
    bg="blue",
    fg="yellow",
    command=get_file())

root.button_2 = tk.Button(root, text="Quit", command=lambda root=root:quit()).pack()

root.mainloop()

C:/Users/admin/Documents/Diablo III/D3Prefs.txt


AttributeError: type object 'Tk' has no attribute 'children'

: 

In [ ]:
import tkinter as tk

def kill(x):
    x.quit()
    x.destroy()
    
def quit(self):
    self.destroy()
    exit()

root = tk.Tk()
button = tk.Button(
    text="Click me!",
    width=25,
    height=50,
    bg="blue",
    fg="yellow",
    command=quit(root)
)
greeting = tk.Label(text="Hello, Tkinter")
greeting.pack()
button.pack()

root.mainloop()

: 

In [ ]:

from tkinter import Tk, Label
root = Tk()
w = Label(root, text="Hello, world!")
w.pack()
root.mainloop()

In [ ]:
import pandas as pd
import numpy as np
import sys 
from tkinter import * 
root = Tk()
root.geometry('580x250')
dates = pd.date_range('20210101', periods=8)
dframe = pd.DataFrame(np.random.randn(8,4),index=dates,columns=list('ABCD'))
txt = Text(root)
txt.pack()

class PrintToTXT(object): 
    def write(self, s):
        txt.insert(END, s)

sys.stdout = PrintToTXT()

print ('Pandas date range of 8 values in 1 timestamp column adjacent to a numpy random float array of 8 rows and 4 columns, displayed in a Tkinter table') 
print (dframe)
root.mainloop()

In [ ]:
import tkinter as tk

window = tk.Tk()

frame_a = tk.Frame()
frame_b = tk.Frame()

label_a = tk.Label(master=frame_a, text="I'm in Frame A")
label_a.pack()

label_b = tk.Label(master=frame_b, text="I'm in Frame B")
label_b.pack()

frame_a.pack()
frame_b.pack()

window.mainloop()


In [ ]:
from tkinter import *
window=tk.Tk()

# add widgets here
v1 = IntVar()
v2 = IntVar()
C1 = Checkbutton(window, text = "Cricket", variable = v1)
C2 = Checkbutton(window, text = "Tennis", variable = v2)
C1.place(x=100, y=100)
C2.place(x=180, y=100)

window.title('Hello Python')
window.geometry("300x200")
window.mainloop()

if v1 == True:
    print('yep')

In [ ]:
from tkinter import *
class MyWindow:
    def __init__(self, win):
        self.lbl1=Label(win, text='First number')
        self.lbl2=Label(win, text='Second number')
        self.lbl3=Label(win, text='Result')
        self.t1=Entry(bd=3)
        self.t2=Entry()
        self.t3=Entry()
        self.btn1 = Button(win, text='Add')
        self.btn2=Button(win, text='Subtract')
        self.lbl1.place(x=100, y=50)
        self.t1.place(x=200, y=50)
        self.lbl2.place(x=100, y=100)
        self.t2.place(x=200, y=100)
        self.b1=Button(win, text='Add', command=self.add)
        self.b2=Button(win, text='Subtract')
        self.b2.bind('<Button-1>', self.sub)
        self.b1.place(x=100, y=150)
        self.b2.place(x=200, y=150)
        self.lbl3.place(x=100, y=200)
        self.t3.place(x=200, y=200)
    def add(self):
        self.t3.delete(0, 'end')
        num1=int(self.t1.get())
        num2=int(self.t2.get())
        result=num1+num2
        self.t3.insert(END, str(result))
    def sub(self, event):
        self.t3.delete(0, 'end')
        num1=int(self.t1.get())
        num2=int(self.t2.get())
        result=num1-num2
        self.t3.insert(END, str(result))

window=Tk()
mywin=MyWindow(window)
window.title('Hello Python')
window.geometry("400x300")
window.mainloop()

In [ ]:
# # Check if columns' list and labels matches -
# # this script would stop otherwise
# if csv_df.columns.all() == google_df.columns.all():
    

#     # Check if our CSV has new rows over GSheets
#     if difference > 0:
#         print("They differ by ", difference, ' strings!')

#     # If yes, we push new lines to GSheets in a cycle
#         current_row = google_len + 1

#         for i in range(difference+1):
#             taken_row = csv_df.iloc[current_row-2].astype('str')
            
#             # constructing an Excel-style pointer
#             diff_range = str('A' + str(current_row) + ':S' + str(current_row))
            
#             # it double checks if'd write into an empty row
#             if google_sheet.sheet1.cell(current_row,1).value == None:
#                 google_sheet.sheet1.update(values =
#                                         [taken_row.values.tolist()],
#                                         range_name = diff_range)
#             current_row+=1
#         print('Tables are in sync.')
    
#     # If it's the oposite or they are equal,
#     # nothing would happen   
#     elif difference < 0:
#         print('Your CSV has less entries than Google Sheets. Wrong SCV?')
#     else: print('Tables are in sync. No actions needed.')

# else: print ('Column labels don\'t match. Wrong tables?')